# Step 0: Initial setup

**NOTE:** 
* Following tasks must be performed each time the notebook instance is started.
* This is **NOT** required when the Kernel is restarted.

**NOTE:** Installation of `jupyter_bokeh` may take a while

In [ ]:
conda install jupyter_bokeh

# Step 1: Basic setup

**NOTE:** Start here when you restarted your Kernel **ONLY**.

## Step 1.1: Imports

In [35]:
import sys
import os
from datetime import datetime
from pathlib import Path

# importing forecast notebook utility from notebooks/common directory
sys.path.insert(0, os.path.abspath("./common/"))
import util
import util.fcst_utils

%reload_ext autoreload
import boto3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams['figure.figsize'] = (15.0, 5.0)

pd.options.mode.copy_on_write = True

from bokeh.io import output_notebook
from bokeh.plotting import show

output_notebook()

Loading BokehJS ...

## Step 1.2: Setup variables

In [13]:
# Resource tag values
owner = "martin.macecek@rearc.io"
type = "Internal"
usage = "Playground"

# Custom variables
prefix = "mac-training"
role_name = f"{prefix}-forecasting"
bucket_name = f"{prefix}-bucket-275279264324-us-east-1"
path = "forecasting/input"
data_key = f"{path}/RIVN.csv"
future_data_key = f"{path}/RIVN_20240513.csv"
prepared_data_key_prefix = "forecasting/prepared/rivn"
item_id = "RIVN"
target_column_name = "close"

# Setup more variables
s3_target_data_key = f"s3://{bucket_name}/{prepared_data_key_prefix}.csv"
s3_related_data_key = f"s3://{bucket_name}/{prepared_data_key_prefix}_rts.csv"
date_format = '%Y%m%d_%H%M%S'
ui_date_format = '%a, %d %b %Y %H:%M:%S %Z'

# Tags for resource tagging
tags = [{'Key': 'Owner', 'Value': owner},
        {'Key': 'Type', 'Value': type},
        {'Key': 'Usage', 'Value': usage}]

## Step 1.3: API connectivity

In [3]:
region = boto3.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
print(f"Account: {account_id}, Region: {region}")

Account: 275279264324, Region: us-east-1


In [4]:
# Connect API sessions
session = boto3.Session(region_name=region) 
s3 = session.client(service_name='s3')
forecast = session.client(service_name='forecast')
forecastquery = session.client(service_name='forecastquery')

## Step 1.4: AWS resources

In [5]:
# Create or retrieve the role to provide to Amazon Forecast.
role_arn = util.get_or_create_iam_role(role_name=role_name)

# echo user inputs without account
print(f"Success! Role '{role_arn.split('/')[1]}' ready for use.")

The role mac-training-forecasting exists, skipping creation...
Done.
Success! Role 'mac-training-forecasting' ready for use.


In [6]:
util.get_or_create_bucket(bucket_name, region=region)
print(f"Success! Bucket '{bucket_name}' ready for use.")

Success! Bucket 'mac-training-bucket-275279264324-us-east-1' ready for use.


# Step 2: Data preparation

In [7]:
stock_df = util.prepare_data(bucket_name, data_key, "%m/%d/%Y", target_column_name, item_id, fill_missing_values=True)
stock_df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,datetime,close,open,high,low,item_id
0,2024-05-13,10.90,10.04,11.19,10.04,RIVN
1,2024-05-12,9.99,10.20,10.40,9.84,RIVN
2,2024-05-11,9.99,10.20,10.40,9.84,RIVN
3,2024-05-10,9.99,10.20,10.40,9.84,RIVN
4,2024-05-09,10.22,10.25,10.47,10.04,RIVN


In [ ]:
stock_df.plot(x='datetime', y=[target_column_name, 'open', 'high', 'low'], figsize=(15, 8))
plt.xlabel('Date Time')
plt.ylabel('Stock Price')
plt.show()

In [59]:
def prepare_data(bucket_name, data_key, date_format, target_column_name, item_id, fill_missing_values=False, minimal=False, start_date=None, end_date=None):
    df = pd.read_csv(f"s3://{bucket_name}/{data_key}", dtype=object)
    df.drop(["Vol.", "Change %"], axis=1, inplace=True)
    df["Date"] = pd.to_datetime(df["Date"], format=date_format).dt.date
    df[["Price", "Open", "High", "Low"]] = df[["Price", "Open", "High", "Low"]].astype(float)
    df.rename(columns={'Date': 'datetime', 'Price': target_column_name, 'Open': 'open', 'High': 'high', 'Low': 'low'}, inplace=True)
    df["item_id"] = item_id
    
    if start_date and end_date:
        mask = (df['datetime'] >= start_date.to_pydatetime().date()) & (df['datetime'] <= end_date.to_pydatetime().date())
        df = df.loc[mask]
    elif start_date and not end_date:
        mask = (df['datetime'] >= start_date.to_pydatetime().date())
        df = df.loc[mask]
    elif not start_date and end_date:
        mask = (df['datetime'] <= end_date.to_pydatetime().date())
        df = df.loc[mask]

    if fill_missing_values:
        new_index = pd.Index(pd.date_range(df.datetime.min(), df.datetime.max()), name="datetime")
        df.set_index("datetime").reindex(new_index)
        df = df.set_index("datetime").reindex(new_index).reset_index().ffill()
        df = df.sort_values(by=['datetime'], ascending=False, ignore_index=True)

    if minimal:
        df.drop(["open", "high", "low"], axis=1, inplace=True)
        df.rename(columns={'datetime': 'timestamp'}, inplace=True)

    return df

In [27]:
def is_excluded(obj: str, exclude: list[str]=[]):
    if not exclude:
        return False
    
    for one_exclude in exclude:
        if one_exclude in obj:
            return True
        
    return False

In [45]:
def get_related_data(s3_client, bucket: str, prefix: str, exclude: list[str]=[], start_date=None, end_date=None):
    object_prefix = prefix if prefix.endswith('/') else f"{prefix}/"
    
    args = {
        "Bucket": bucket,
        "Prefix": object_prefix
    }
    related_data_csv_objects = {}
    while True:
        response = s3_client.list_objects_v2(**args)
        list_related_data_csv_objects = list(filter(lambda obj: obj != object_prefix and not is_excluded(obj, exclude), list(map(lambda obj: obj["Key"], response["Contents"]))))
        related_data_csv_objects = { Path(obj).stem.lower() : obj for obj in list_related_data_csv_objects}
        if "NextContinuationToken" in response:
            args["ContinuationToken"] = response["NextContinuationToken"]
        else:
            break
    
    related_stock_dfs = []
    for key in related_data_csv_objects:
        related_stock_dfs.append(
            prepare_data(
                bucket,
                related_data_csv_objects[key],
                "%m/%d/%Y",
                target_column_name,
                key,
                fill_missing_values=True,
                start_date=start_date,
                end_date=end_date
            )
        )
    
    return related_stock_dfs

In [60]:
get_related_data(s3, bucket_name, path, [data_key, future_data_key], start_date=stock_df.datetime.min(), end_date=stock_df.datetime.max())

[      datetime   close    open    high     low item_id
 0   2024-05-13  186.57  188.00  188.18  185.36    amzn
 1   2024-05-12  187.48  189.16  189.87  186.93    amzn
 2   2024-05-11  187.48  189.16  189.87  186.93    amzn
 3   2024-05-10  187.48  189.16  189.87  186.93    amzn
 4   2024-05-09  189.50  188.88  191.68  187.52    amzn
 ..         ...     ...     ...     ...     ...     ...
 910 2021-11-15  177.28  176.85  179.69  176.29    amzn
 911 2021-11-14  176.26  174.25  177.04  172.35    amzn
 912 2021-11-13  176.26  174.25  177.04  172.35    amzn
 913 2021-11-12  176.26  174.25  177.04  172.35    amzn
 914 2021-11-11  173.62  175.65  177.16  173.37    amzn
 
 [915 rows x 6 columns],
       datetime  close   open   high    low item_id
 0   2024-05-13  12.32  12.09  12.45  12.09    ford
 1   2024-05-12  11.99  12.14  12.17  11.92    ford
 2   2024-05-11  11.99  12.14  12.17  11.92    ford
 3   2024-05-10  11.99  12.14  12.17  11.92    ford
 4   2024-05-09  12.14  12.12  12.21  12.

# Step 3: Prepare and Save the Target Time Series

In this exmple, we are only using the supplemental fields `open`, `high` and `low` for forecasting.
Later we may explore with supplemental values from other stocks.

In [ ]:
# Forecast length in days (Units is defined below)
FORECAST_LENGTH = 30

# What is your forecast time unit granularity?
# Choices are: ^Y|M|W|D|h|30min|15min|10min|5min|1min$ 
DATASET_FREQUENCY = "D"
TIMESTAMP_FORMAT = "yyyy-MM-dd"
# delimiter = ','

# What name do you want to give this project?
# We will use this same name for your Forecast Dataset Group name.
PROJECT = 'rivn-forecast'
DATA_VERSION = 1

In [ ]:
target_df = stock_df[['item_id', 'datetime', target_column_name]][:-FORECAST_LENGTH]
target_df.head(5)

In [ ]:
rts_df = stock_df[['item_id', 'datetime', 'open', 'high', 'low']]
rts_df.head(5)

In [ ]:
print(f"{len(target_df)} + {FORECAST_LENGTH} = {len(rts_df)}")
assert len(target_df) + FORECAST_LENGTH == len(rts_df), "length doesn't match"

In [ ]:
target_df.to_csv(s3_target_data_key, index= False, header = False)
rts_df.to_csv(s3_related_data_key, index= False, header = False)

# Step 4: Create the Dataset Group and Dataset

## Dataset Group

In [ ]:
dataset_group_name = f"{prefix}_{PROJECT}_{DATA_VERSION}".replace("-", "_")
print(f"Dataset Group Name = {dataset_group_name}")

In [ ]:
dataset_arns = []
try:
    create_dataset_group_response = \
        forecast.create_dataset_group(Domain="RETAIL",
                                      DatasetGroupName=dataset_group_name,
                                      DatasetArns=dataset_arns,
                                      Tags=tags
                                     )
    dataset_group_arn = create_dataset_group_response['DatasetGroupArn']
    status = util.wait(lambda: forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn))
    assert status
except forecast.exceptions.ResourceAlreadyExistsException:
    dataset_group_arn = f"arn:aws:forecast:{region}:{account_id}:dataset-group/{dataset_group_name}"
    print(f"Dataset group {dataset_group_arn} already exists.")

## Target Schema

In [ ]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
tts_schema = {
   "Attributes": [
      {
         "AttributeName": "item_id",
         "AttributeType": "string"
      },
      {
         "AttributeName": "timestamp",
         "AttributeType": "timestamp"
      },
      {
         "AttributeName": "demand",
         "AttributeType": "float"
      }
   ]
}

## Target Dataset

In [ ]:
tts_dataset_name = f"{dataset_group_name}_tts"
print(tts_dataset_name)

In [ ]:
try:
    create_dataset_tts_response = \
        forecast.create_dataset(Domain="RETAIL",
                                DatasetType='TARGET_TIME_SERIES',
                                DatasetName=tts_dataset_name,
                                DataFrequency=DATASET_FREQUENCY,
                                Schema=tts_schema,
                                Tags=tags
                               )
    tts_dataset_arn = create_dataset_tts_response['DatasetArn']
    status = util.wait(lambda: forecast.describe_dataset(DatasetArn=tts_dataset_arn))
    assert status
except forecast.exceptions.ResourceAlreadyExistsException:
    tts_dataset_arn = f"arn:aws:forecast:{region}:{account_id}:dataset/{tts_dataset_name}"
    print(f"Target dataset {tts_dataset_arn} already exists.")

## Related schema

In [ ]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
rts_schema = {
   "Attributes": [
      {
         "AttributeName": "item_id",
         "AttributeType": "string"
      },
      {
         "AttributeName": "timestamp",
         "AttributeType": "timestamp"
      },
      {
         "AttributeName": "open_price",
         "AttributeType": "float"
      },
      {
         "AttributeName": "high_price",
         "AttributeType": "float"
      },
      {
         "AttributeName": "low_price",
         "AttributeType": "float"
      }
   ]
}

## Related dataset

In [ ]:
rts_dataset_name = f"{dataset_group_name}_rts"
print(rts_dataset_name)

In [ ]:
try:
    create_dataset_rts_response = \
        forecast.create_dataset(Domain="RETAIL",
                                DatasetType='RELATED_TIME_SERIES',
                                DatasetName=rts_dataset_name,
                                DataFrequency=DATASET_FREQUENCY,
                                Schema=rts_schema,
                                Tags=tags
                               )
    rts_dataset_arn = create_dataset_rts_response['DatasetArn']
    status = util.wait(lambda: forecast.describe_dataset(DatasetArn=rts_dataset_arn))
    assert status
except forecast.exceptions.ResourceAlreadyExistsException:
    rts_dataset_arn = f"arn:aws:forecast:{region}:{account_id}:dataset/{rts_dataset_name}"
    print(f"Related dataset {rts_dataset_arn} already exists.")

In [ ]:
dataset_arns = []
dataset_arns.append(tts_dataset_arn)
dataset_arns.append(rts_dataset_arn)
update_dataset_response = forecast.update_dataset_group(DatasetGroupArn=dataset_group_arn, DatasetArns=dataset_arns)
status = util.wait(lambda: forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn))
assert status

# Step 5: Import data from S3 to Forecast

## Target data

**Note:** Depending on the data size, the import can take 10 mins or more to become **ACTIVE**.

In [ ]:
import_tts_dataset_response = True
if len(util.get_dataset_import_jobs(tts_dataset_arn, forecast)) > 0:
    print("Target dataset has already imported data.")
    import_tts_dataset_response = True if input("Re-import (y/N)? ").lower() == "y" else False

if import_tts_dataset_response:
    tts_dataset_import_job_response = \
        forecast.create_dataset_import_job(DatasetImportJobName=f"tts_job_{datetime.now().strftime(date_format)}",
                                           DatasetArn=tts_dataset_arn,
                                           DataSource= {
                                             "S3Config" : {
                                                 "Path": s3_target_data_key,
                                                 "RoleArn": role_arn
                                             } 
                                           },
                                           TimestampFormat=TIMESTAMP_FORMAT,
                                           Tags=tags
                                          )
    tts_dataset_import_job_arn=tts_dataset_import_job_response['DatasetImportJobArn']
    status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=tts_dataset_import_job_arn))
    assert status
    print("Target data imported.")
else:
    print("Target data re-import skipped")

## Related data

**Note:** Depending on the data size, the import can take 10 mins or more to become **ACTIVE**.

In [ ]:
import_rts_dataset_response = True
if len(util.get_dataset_import_jobs(rts_dataset_arn, forecast)) > 0:
    print("Related dataset has already imported data.")
    import_rts_dataset_response = True if input("Re-import (y/N)? ").lower() == "y" else False

if import_rts_dataset_response:
    rts_dataset_import_job_response = \
        forecast.create_dataset_import_job(DatasetImportJobName=f"rts_job_{datetime.now().strftime(date_format)}",
                                           DatasetArn=rts_dataset_arn,
                                           DataSource= {
                                             "S3Config" : {
                                                 "Path": s3_related_data_key,
                                                 "RoleArn": role_arn
                                             } 
                                           },
                                           TimestampFormat=TIMESTAMP_FORMAT,
                                           Tags=tags
                                          )
    rts_dataset_import_job_arn = rts_dataset_import_job_response["DatasetImportJobArn"]
    status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=rts_dataset_import_job_arn))
    assert status
    print("Related data imported.")
else:
    print("Related data re-import skipped")

# Step 6: Training and Evaluation

## DeepAR+

In [ ]:
algorithm_arn = 'arn:aws:forecast:::algorithm/'
algorithm = 'Deep_AR_Plus'
algorithm_arn_deep_ar_plus = algorithm_arn + algorithm
predictor_name_deep_ar = f"{dataset_group_name}_{algorithm.lower()}"
print(f"Predictor Name = {predictor_name_deep_ar}")

**NOTE:** Training a forecast model can take several hours to become **ACTIVE**.

In [ ]:
retrain_predictor_deep_ar = False
existing_predictor_deep_ar = util.get_predictor(predictor_name_deep_ar, forecast)

if existing_predictor_deep_ar:
    predictor_arn_deep_ar = existing_predictor_deep_ar['PredictorArn']
    print(f"DeepAR+ Predictor {predictor_arn_deep_ar} already exists.")
    retrain_predictor_deep_ar = True if input("Retrain model (y/N)? ").lower() == "y" else False

if existing_predictor_deep_ar and retrain_predictor_deep_ar:
    util.delete_forecasts_by_predictor(predictor_arn_deep_ar, forecast)
    print(f"Deleting DeepAR+ Predictor {predictor_arn_deep_ar}...")
    util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=predictor_arn_deep_ar))
elif existing_predictor_deep_ar and not retrain_predictor_deep_ar:
    print(f"Keeping existing DeepAR+ Predictor {predictor_arn_deep_ar}.")

if not existing_predictor_deep_ar or retrain_predictor_deep_ar:
    create_predictor_deep_ar_response = \
        forecast.create_predictor(PredictorName=predictor_name_deep_ar,
                                  AlgorithmArn=algorithm_arn_deep_ar_plus,
                                  ForecastHorizon=FORECAST_LENGTH,
                                  PerformAutoML=False,
                                  PerformHPO=False,
                                  InputDataConfig={
                                      "DatasetGroupArn": dataset_group_arn,
                                      "SupplementaryFeatures": [
                                          {"Name": "holiday",
                                           "Value": "US"}],
                                  },
                                  FeaturizationConfig={"ForecastFrequency": DATASET_FREQUENCY},
                                  Tags=tags
                                 )
    predictor_arn_deep_ar = create_predictor_deep_ar_response['PredictorArn']
    print(f"Creating DeepAR+ Predictor {predictor_arn_deep_ar}...")
    status = util.wait(lambda: forecast.describe_predictor(PredictorArn=predictor_arn_deep_ar))
    assert status

## Prophet

In [ ]:
algorithm_arn = 'arn:aws:forecast:::algorithm/'
algorithm = 'Prophet'
algorithm_arn_prophet = algorithm_arn + algorithm
predictor_name_prophet = f"{dataset_group_name}_{algorithm.lower()}"
print(f"Predictor Name = {predictor_name_prophet}")

**NOTE:** Training a forecast model can take several hours to become **ACTIVE**.

In [ ]:
retrain_predictor_prophet = False
existing_predictor_prophet = util.get_predictor(predictor_name_prophet, forecast)

if existing_predictor_prophet:
    predictor_arn_prophet = existing_predictor_prophet['PredictorArn']
    print(f"Prophet Predictor {predictor_arn_prophet} already exists.")
    retrain_predictor_prophet = True if input("Retrain model (y/N)? ").lower() == "y" else False

if existing_predictor_prophet and retrain_predictor_prophet:
    print(f"Deleting Prophet Predictor {predictor_arn_prophet}...")
    util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=predictor_arn_prophet))
elif existing_predictor_prophet and not retrain_predictor_prophet:
    print(f"Keeping existing Prophet Predictor {predictor_arn_prophet}.")

if not existing_predictor_prophet or retrain_predictor_prophet:
    create_predictor_response = \
        forecast.create_predictor(PredictorName=predictor_name_prophet,
                                  AlgorithmArn=algorithm_arn_prophet,
                                  ForecastHorizon=FORECAST_LENGTH,
                                  PerformAutoML=False,
                                  PerformHPO=False,
                                  InputDataConfig= {
                                      "DatasetGroupArn": dataset_group_arn,
                                      "SupplementaryFeatures": [
                                          {"Name": "holiday",
                                           "Value": "US"}],
                                  },
                                  FeaturizationConfig={"ForecastFrequency": DATASET_FREQUENCY},
                                  Tags=tags
                                 )
    predictor_arn_prophet = create_predictor_response['PredictorArn']
    print(f"Creating Prophet Predictor {predictor_arn_prophet}...")
    status = util.wait(lambda: forecast.describe_predictor(PredictorArn=predictor_arn_prophet))
    assert status

## Auto

In [ ]:
predictor_name_auto = f"{dataset_group_name}_auto"
print(f"Predictor Name = {predictor_name_auto}")

**NOTE:** Training a forecast model can take several hours to become **ACTIVE**.

In [ ]:
existing_predictor_auto = util.get_predictor(predictor_name_auto, forecast)
if existing_predictor_auto:
    predictor_arn_auto = existing_predictor_auto['PredictorArn']
    print(f"Auto Predictor {predictor_arn_auto} already exists.")
    if input("Retrain model (y/N)? ").lower() == "y":
        args = {
            "PredictorName": f"{predictor_name_auto}_retrain_{datetime.now().strftime(date_format)}",
            "ReferencePredictorArn": predictor_arn_auto
        }
    else:
        args = {}
else:
    args = {
        "PredictorName": predictor_name_auto,
        "ForecastHorizon": FORECAST_LENGTH,
        "ForecastFrequency": DATASET_FREQUENCY,
        "DataConfig": {
            "DatasetGroupArn": dataset_group_arn,
            "AdditionalDatasets": [
                {
                    "Name": "holiday",
                    "Configuration": {
                        "CountryCode": ["US"]
                    }
                }
            ]
        },
        "Tags": tags
    }

if args:
    create_predictor_auto_response = forecast.create_auto_predictor(**args)
    predictor_arn_auto = create_predictor_auto_response['PredictorArn']
    print(f"{'Retraining existing' if existing_predictor_auto else 'Creating new'} Auto Predictor {predictor_arn_auto}...")
    status = util.wait(lambda: forecast.describe_auto_predictor(PredictorArn=predictor_arn_auto))
    assert status
else:
    print(f"Keeping existing Auto Predictor {predictor_arn_auto}.")

## Step 7: Predictor Error Metrics

In [ ]:
error_metrics_deep_ar_plus = forecast.get_accuracy_metrics(PredictorArn=predictor_arn_deep_ar)

In [ ]:
error_metrics_prophet = forecast.get_accuracy_metrics(PredictorArn=predictor_arn_prophet)

In [ ]:
error_metrics_auto = forecast.get_accuracy_metrics(PredictorArn=predictor_arn_auto)

In [ ]:
def extract_summary_metrics(metric_response, predictor_name):
    df = pd.DataFrame(metric_response['PredictorEvaluationResults']
                 [0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses'])
    df['Predictor'] = predictor_name
    return df

In [ ]:
deep_ar_metrics = extract_summary_metrics(error_metrics_deep_ar_plus, "DeepAR")
prophet_metrics = extract_summary_metrics(error_metrics_prophet, "Prophet")
auto_metrics = extract_summary_metrics(error_metrics_auto, "Auto")

In [ ]:
pd.concat([deep_ar_metrics, prophet_metrics, auto_metrics]) \
    .pivot(index='Quantile', columns='Predictor', values='LossValue').plot.bar()

# Step 8: Forecasting

## DeepAR+

**NOTE:** Creating a forecast can take up to an hour to become **ACTIVE**.

In [ ]:
forecast_arn_deep_ar = util.create_forecast(
    *util.create_forecast_name(dataset_group_name, 
                               'deep_ar',
                               date_format=date_format
                              ),
    predictor_arn_deep_ar,
    forecast,
    tags,
    ui_date_format=ui_date_format,
)

## Prophet

**NOTE:** Creating a forecast can take up to an hour to become **ACTIVE**.

In [ ]:
forecast_arn_prophet = util.create_forecast(
    *util.create_forecast_name(dataset_group_name, 
                               'prophet',
                               date_format=date_format
                              ),
    predictor_arn_prophet,
    forecast,
    tags,
    ui_date_format=ui_date_format,
)

## Auto

**NOTE:** Creating a forecast can take up to an hour to become **ACTIVE**.

In [ ]:
forecast_arn_auto = util.create_forecast(
    *util.create_forecast_name(dataset_group_name, 
                               'auto',
                               date_format=date_format
                              ),
    predictor_arn_auto,
    forecast,
    tags,
    ui_date_format=ui_date_format,
)

# Step 9: Querying

In [ ]:
exact = util.load_exact_sol(s3_target_data_key, item_id, target_col_name=target_column_name)
exact

In [ ]:
future_df = prepare_data(bucket_name, future_data_key, "%m/%d/%Y", target_column_name, fill_missing_values=True, minimal=True)
future_df

## DeepAR+

In [ ]:
forecast_response_deep_ar = forecastquery.query_forecast(
    ForecastArn=forecast_arn_deep_ar,
    Filters={"item_id": item_id})

In [ ]:
forecast_response_deep_ar

### Plot using default plot

In [ ]:
util.plot_forecasts(
    forecast_response_deep_ar,
    exact,
    freq=f'1{DATASET_FREQUENCY}',
    forecastHorizon=FORECAST_LENGTH,
    time_back=30,
    future=future_df,
    target_col_name=target_column_name,
    reverse=True
)
plt.title("DeepAR Forecast")

### Plot using bokeh

In [ ]:
deep_ar_plot = util.plot_bokeh_forecasts(
    forecast_response_deep_ar,
    exact,
    freq=f'1{DATASET_FREQUENCY}',
    forecastHorizon=FORECAST_LENGTH,
    time_back=30,
    future=future_df,
    target_col_name=target_column_name,
    reverse=True
)
show(deep_ar_plot)

## Prophet

In [ ]:
forecast_response_prophet = forecastquery.query_forecast(
    ForecastArn=forecast_arn_prophet,
    Filters={"item_id": item_id})

In [ ]:
forecast_response_prophet

### Plot using default plot

In [ ]:
util.plot_forecasts(
    forecast_response_prophet,
    exact,
    freq=f'1{DATASET_FREQUENCY}',
    forecastHorizon=FORECAST_LENGTH,
    time_back=30,
    future=future_df,
    target_col_name=target_column_name,
    reverse=True
)
plt.title("Prophet Forecast")

### Plot using bokeh

In [ ]:
prophet_plot = util.plot_bokeh_forecasts(
    forecast_response_prophet,
    exact,
    freq=f'1{DATASET_FREQUENCY}',
    forecastHorizon=FORECAST_LENGTH,
    time_back=30,
    future=future_df,
    target_col_name=target_column_name,
    reverse=True
)
show(prophet_plot)

## Auto

In [ ]:
forecast_response_auto = forecastquery.query_forecast(
    ForecastArn=forecast_arn_auto,
    Filters={"item_id": item_id})

In [ ]:
forecast_response_auto

### Plot using default plot

In [ ]:
util.plot_forecasts(
    forecast_response_auto,
    exact,
    freq=f'1{DATASET_FREQUENCY}',
    forecastHorizon=FORECAST_LENGTH,
    time_back=30,
    future=future_df,
    target_col_name=target_column_name,
    reverse=True
)
plt.title("Auto Forecast")

### Plot using bokeh

In [ ]:
auto_plot = util.plot_bokeh_forecasts(
    forecast_response_auto,
    exact,
    freq=f'1{DATASET_FREQUENCY}',
    forecastHorizon=FORECAST_LENGTH,
    time_back=30,
    future=future_df,
    target_col_name=target_column_name,
    reverse=True
)
show(auto_plot)

# Cleanup

## Datasets and Dataset Group

In [ ]:
# util.delete_dataset_group(dataset_group_arn, forecast)
util.delete_dataset_group("arn:aws:forecast:us-east-1:275279264324:dataset-group/mac_training_rivn_forecast_1", forecast)